# This notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


I need the package wikitables to extract the date from the wikipedia

In [3]:
import sys
!{sys.executable} -m pip install wikitables

Requirement not upgraded as not directly required: requests>=2.9.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from wikitables)
    100% |████████████████████████████████| 133kB 5.4MB/s eta 0:00:01
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.9.1->wikitables)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.9.1->wikitables)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.9.1->wikitables)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.9.1->wikitables)
  Running setup.py bdist_wheel for mwparserfromhell ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/5a/ef/c

1. Treat the data

In [4]:
from wikitables import import_tables
tables = import_tables('List of postal codes of Canada: M')

In [5]:
neighbourhoods_data=tables[0].rows

In [6]:
# define the dataframe columns
column_names = [] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

I fill the empty dataframe created with the data extracted and I create the three columns required

In [7]:
for data in neighbourhoods_data:
    postal_code=data['Postcode']
    borough_name = data['Borough'] 
    neighbourhood_name = data['Neighbourhood']
    neighbourhoods = neighbourhoods.append({'PostalCode':postal_code, 'Borough': borough_name, 'Neighbourhood':neighbourhood_name}, ignore_index=True)

In [8]:
neighbourhoods.head()

,Borough,Neighbourhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


I desestimate the 'Borough' rows with values 'Not assigned'

In [9]:
df = neighbourhoods.applymap(str)
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.head(10)

,Borough,Neighbourhood,PostalCode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B


I group the Neighbourhood column by PostalCode equal values 

In [10]:
df1=df.groupby('PostalCode').agg(lambda x: ', '.join(set(x)))

In [11]:
df1.head(10)

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"


I replace the Neighbourhood values 'Not assigned' by the Borough values

In [12]:
df1.loc[df1['Neighbourhood']=='Not assigned','Neighbourhood']=df1.loc[df1['Neighbourhood']=='Not assigned','Borough']

Number of rows below:

In [13]:
df1.shape

(103, 2)

In [14]:
df2=pd.read_csv("http://cocl.us/Geospatial_data")
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [16]:
result = pd.concat([df1, df2], axis=1)

In [17]:
result.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Port Union, Rouge Hill, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


2. Explore the data

Let's create a function to repeat the same process to all the neighbourhoods in Toronto

In [18]:
CLIENT_ID = '5I30MDOXKVLKCQURREIZ0FSWHABE14LI4C35JQV2S5R0K3P2' # your Foursquare ID
CLIENT_SECRET = '3ST1EJE1CK5CWLTXQ2H0WZLPBHWKKFMW1ZG5N1FYJHPLI1TH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [19]:
import requests

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We run the function in our data...

In [21]:
toronto_venues = getNearbyVenues(names=result['Neighbourhood'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

Rouge, Malvern
Port Union, Rouge Hill, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview, Kennedy Park, East Birchmount Park
Golden Mile, Oakridge, Clairlea
Cliffside, Scarborough Village West, Cliffcrest
Cliffside West, Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Sullivan, Clarks Corners, Tam O'Shanter
Milliken, L'Amoreaux East, Agincourt North, Steeles East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Downsview North, Bathurst Manor, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

We check the size...

In [22]:
print(toronto_venues.shape)

(1322, 7)


Let's check how many venues were returned for each neighborhood

In [23]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,3,3,3,3,3,3
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,30,30,30,30,30,30
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CFB Toronto, Downsview East",4,4,4,4,4,4
"CN Tower, Railway Lands, Harbourfront West, Island airport, South Niagara, Bathurst Quay, King and Spadina",13,13,13,13,13,13
"Cabbagetown, St. James Town",30,30,30,30,30,30
Caledonia-Fairbanks,6,6,6,6,6,6


3. Analyze each Neighbourhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Port Union, Rouge Hill, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(1322, 236)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5,"CFB Toronto, Downsview East",0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,"CN Tower, Railway Lands, Harbourfront West, Is...",0.0,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
7,"Cabbagetown, St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,Caledonia-Fairbanks,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000
9,Canada Post Gateway Processing Centre,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0          Lounge  0.33
1  Breakfast Spot  0.33
2  Sandwich Place  0.33
3       Pet Store  0.00
4  Massage Studio  0.00


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.07
1    Italian Restaurant  0.07
2  Fast Food Restaurant  0.07
3         Grocery Store  0.04
4              Boutique  0.04


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.10
1      Farmers Market  0.07
2  Seafood Restaurant  0.07
3              Bakery  0.07
4                Café  0.07


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1                  Park  0.06
2         Garden Center  0.06
3            Smoke Shop  0.06
4  Fast Food R

Let's put that into a pandas dataframe

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Yoga Studio,Cocktail Bar,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
1,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Cosmetics Shop,Pharmacy,Café,Pizza Place,Butcher,Pub,Sushi Restaurant
3,Berczy Park,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Café,Italian Restaurant,Basketball Stadium,Beer Bar,Jazz Club,Bistro
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Skate Park,Brewery,Fast Food Restaurant,Auto Workshop,Farmers Market,Butcher,Spa,Pizza Place,Burrito Place
5,"CFB Toronto, Downsview East",Park,Airport,Playground,Bus Stop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Electronics Store,Curling Ice,Discount Store
6,"CN Tower, Railway Lands, Harbourfront West, Is...",Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Boutique,Harbor / Marina,Sculpture Garden,Airport Gate,Airport Food Court,Airport
7,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Italian Restaurant,Café,General Entertainment,Market,Indian Restaurant,Beer Store,Jewelry Store,Diner
8,Caledonia-Fairbanks,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant,Curling Ice,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
9,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Gym / Fitness Center,Mediterranean Restaurant,Sandwich Place,Burrito Place,Middle Eastern Restaurant,Fried Chicken Joint,American Restaurant,Dumpling Restaurant


4. Cluster Neighbourhoods

In [30]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  44.00 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.50 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.57 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  35.36 MB/s
Libraries imported.


Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 0, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [40]:
print (toronto_merged)

              Borough                                      Neighbourhood  \
0         Scarborough                                     Rouge, Malvern   
1         Scarborough             Rouge Hill, Highland Creek, Port Union   
2         Scarborough                  West Hill, Guildwood, Morningside   
3         Scarborough                                             Woburn   
4         Scarborough                                          Cedarbrae   
5         Scarborough                                Scarborough Village   
6         Scarborough        Kennedy Park, Ionview, East Birchmount Park   
7         Scarborough                    Golden Mile, Oakridge, Clairlea   
8         Scarborough    Scarborough Village West, Cliffside, Cliffcrest   
9         Scarborough                        Cliffside West, Birch Cliff   
10        Scarborough  Dorset Park, Wexford Heights, Scarborough Town...   
11        Scarborough                                  Maryvale, Wexford   
12        Sc

In [70]:
toronto_merged = result

# add clustering labels

toronto_merged['Cluster Labels'] = kmeans.labels_[5]

#print(len(labels))

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,3,Fast Food Restaurant,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,Scarborough,"Port Union, Rouge Hill, Highland Creek",M1C,43.784535,-79.160497,3,Bar,Yoga Studio,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,3,Electronics Store,Rental Car Location,Spa,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Drugstore,Dog Run,Discount Store
3,Scarborough,Woburn,M1G,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,3,Athletics & Sports,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Bakery,Caribbean Restaurant,Dessert Shop,Dim Sum Restaurant


Let's visualize the resulting clusters...

In [58]:
from geopy.geocoders import Nominatim

In [59]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [71]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

5. Explore clustering

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
